## Barra transitoria con disipación

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ------ Propiedades barra --------
rho = 2700  # kg/m3
c = 900 # J/kgK
k = 180 # W/mK
aph = k/(rho*c)
L = 1.0 # m
W = 0.1 # m
H = 0.1 # m
A = W*H # m2
qv = 1000/(A*L) # W/m3
# ----------------------------------


# ------ Malla --------
Nx = 10
Nt = 100
time = 5000 # s

dx = L/(Nx-1)
dt = time/Nt
# ----------------------


# ------ Solución numérica Euler explícito --------
T_vector = np.zeros((Nx, Nt+1))

T_vector[:, 0] = 0 # Condición inicial
T_vector[0, :] = 0 # Condición de frontera izquierda
T_vector[-1, :] = 0 # Condición de frontera derecha

Fo = aph*dt/dx**2
for it in range(1, Nt+1):
    for ix in range(1,Nx-1):
        T_vector[ix, it] = T_vector[ix, it-1] + Fo * (T_vector[ix+1, it-1] - 2*T_vector[ix, it-1] + T_vector[ix-1, it-1]) + qv*dt/(rho*c)

# -------------------------------------------------


# ------ Plot --------

x = np.linspace(0, L, Nx)
# Plot cada 1000s independientemente del paso de tiempo
idx_1000s = int(1000/dt)
for it in range(0, Nt+1, idx_1000s):
    plt.plot(x, T_vector[:, it], label=f't={it*dt:.1f}s')

plt.xlabel('x (m)')
plt.ylabel('T (°C)')
plt.legend()
plt.title('Distribución de temperatura de la barra')
plt.show()

print(f'Fourier number ={Fo:.3f}')
# Pintar la temperatura en el centro de la barra a lo largo del tiempo
t = np.linspace(0, time, Nt+1)
plt.plot(t, T_vector[Nx//2, :])
plt.xlabel('t (s)')
plt.ylabel('T (°C)')
plt.title('Evolución de temperatura en el centro de la barra')
plt.show()
# ------------------

## Barra con convección y radiación

In [ ]:
# -------- Convección y radiación --------
h = 10 # W/m2K
eps = 0.9
sigma = 5.67e-8 # W/m2K4
T_inf = 0 # °C
p = 2*(W+H) # perimetro
conv_gl = h*p/A
rad_gr = eps*sigma*p/A 
#-----------------------------------------


#qv = 3000/(A*L) # W/m3


# ------ Solución numérica Euler explícito con convección y radiación --------
T_conv = np.zeros((Nx, Nt+1))
T_conv[:, 0] = 0 # Condición inicial
T_conv[0, :] = 0 # Condición de frontera izquierda
T_conv[-1, :] = 0 # Condición de frontera derecha

T_rad = np.copy(T_conv)
T_conv_rad = np.copy(T_conv)

for it in range(1, Nt+1):
    for ix in range(1,Nx-1):
        T_conv[ix, it] = T_conv[ix, it-1] + Fo * (T_conv[ix+1, it-1] - 2*T_conv[ix, it-1] + T_conv[ix-1, it-1]) + qv*dt/(rho*c) - conv_gl*dt/(rho*c)*(T_conv[ix, it-1]-T_inf)
        T_rad[ix, it] = T_rad[ix, it-1] + Fo * (T_rad[ix+1, it-1] - 2*T_rad[ix, it-1] + T_rad[ix-1, it-1]) + qv*dt/(rho*c) - rad_gr*dt/(rho*c)*((T_rad[ix, it-1]+273.15)**4 - (T_inf+273.15)**4)
        T_conv_rad[ix, it] = T_conv_rad[ix, it-1] + Fo * (T_conv_rad[ix+1, it-1] - 2*T_conv_rad[ix, it-1] + T_conv_rad[ix-1, it-1]) + qv*dt/(rho*c) - (conv_gl+rad_gr)*dt/(rho*c)*(T_conv_rad[ix, it-1]-T_inf) - rad_gr*dt/(rho*c)*((T_conv_rad[ix, it-1]+273.15)**4 - (T_inf+273.15)**4)

# -----------------------------------------------------------------------------


# ------ Plot comparativo --------
# Temperatura en el centro de la barra a lo largo del tiempo
plt.plot(t, T_vector[Nx//2, :], label='Sin convección ni radiación')
plt.plot(t, T_conv[Nx//2, :], label='Convección')
plt.plot(t, T_rad[Nx//2, :], label='Radiación')
plt.plot(t, T_conv_rad[Nx//2, :], label='Convección y radiación')
plt.xlabel('t (s)')
plt.ylabel('T (°C)')
plt.title('Evolución de temperatura en el centro de la barra')
plt.legend()
plt.show()
# ------------------------------


## Termostato

In [ ]:
# Añadiendo un termostato que mantenga la temperatura en el centro de la barra a 20 °C
qv_heater_barra = 1000/(A*L) # W/m3
T_setpoint = 20 # °C
histeresis = 1 # °C

T_termostato = np.zeros((Nx, Nt+1))
T_termostato[:, 0] = 0 # Condición inicial
T_termostato[0, :] = 0 # Condición de frontera izquierda
T_termostato[-1, :] = 0 # Condición de frontera derecha

for it in range(1, Nt+1):
    if T_termostato[Nx//2, it-1] < T_setpoint - histeresis:
        qv = qv_heater_barra
    elif T_termostato[Nx//2, it-1] > T_setpoint + histeresis:
        qv = 0
    for ix in range(1,Nx-1):
        T_termostato[ix, it] = T_termostato[ix, it-1] + Fo * (T_termostato[ix+1, it-1] - 2*T_termostato[ix, it-1] + T_termostato[ix-1, it-1]) + qv*dt/(rho*c) - (conv_gl+rad_gr)*dt/(rho*c)*(T_termostato[ix, it-1]-T_inf) - rad_gr*dt/(rho*c)*((T_termostato[ix, it-1]+273.15)**4 - (T_inf+273.15)**4)


# Temperatura en el centro de la barra a lo largo del tiempo
plt.plot(t, T_vector[Nx//2, :], label='Disiapación constante')
plt.plot(t, T_termostato[Nx//2, :], label='Con termostato')
plt.xlabel('t (s)')
plt.ylabel('T (°C)')
plt.title('Control por termostato en el centro de la barra')
plt.legend()
plt.show()



## Control PI

In [ ]:
# Control PI con setpoint de 20 °C
Kp = 50000 # Proporcional
Ki = 10 # Integral


T_PI = np.zeros((Nx, Nt+1))
T_PI[:, 0] = 0 # Condición inicial
T_PI[0, :] = 0 # Condición de frontera izquierda
T_PI[-1, :] = 0 # Condición de frontera derecha
error_integral = 0
for it in range(1, Nt+1):
    error = T_setpoint - T_PI[Nx//2, it-1]
    error_integral += error * dt
    qv = Kp * error + Ki * error_integral
    qv = max(0, min(qv, qv_heater_barra)) # Limitar qv entre 0 y qv_heater_barra
    for ix in range(1,Nx-1):
        T_PI[ix, it] = T_PI[ix, it-1] + Fo * (T_PI[ix+1, it-1] - 2*T_PI[ix, it-1] + T_PI[ix-1, it-1]) + qv*dt/(rho*c) - (conv_gl+rad_gr)*dt/(rho*c)*(T_PI[ix, it-1]-T_inf) - rad_gr*dt/(rho*c)*((T_PI[ix, it-1]+273.15)**4 - (T_inf+273.15)**4)

# Temperatura en el centro de la barra a lo largo del tiempo
plt.plot(t, T_vector[Nx//2, :], label='Disiapación constante')
plt.plot(t, T_PI[Nx//2, :], label='Control PI')
plt.xlabel('t (s)')
plt.ylabel('T (°C)')
plt.title('Control PI en el centro de la barra')
plt.legend()
plt.show()



In [ ]:
# Plot interactivo para jugar con Kp y Ki
import ipywidgets as widgets
from IPython.display import display

def update_plot(Kp, Ki):
    T_PI = np.zeros((Nx, Nt+1))
    T_PI[:, 0] = 0 # Condición inicial
    T_PI[0, :] = 0 # Condición de frontera izquierda
    T_PI[-1, :] = 0 # Condición de frontera derecha
    error_integral = 0
    for it in range(1, Nt+1):
        error = T_setpoint - T_PI[Nx//2, it-1]
        error_integral += error * dt
        qv = Kp * error + Ki * error_integral
        qv = max(0, min(qv, qv_heater_barra)) # Limitar qv entre 0 y qv_heater_barra
        for ix in range(1,Nx-1):
            T_PI[ix, it] = T_PI[ix, it-1] + Fo * (T_PI[ix+1, it-1] - 2*T_PI[ix, it-1] + T_PI[ix-1, it-1]) + qv*dt/(rho*c) - (conv_gl+rad_gr)*dt/(rho*c)*(T_PI[ix, it-1]-T_inf) - rad_gr*dt/(rho*c)*((T_PI[ix, it-1]+273.15)**4 - (T_inf+273.15)**4)

    plt.figure(figsize=(10, 5))
    plt.plot(t, T_PI[Nx//2, :], label='Control PI')
    plt.xlabel('t (s)')
    plt.ylabel('T (°C)')
    plt.title(f'Control PI: Kp={Kp}, Ki={Ki}')
    plt.ylim(0, 50)
    plt.grid()
    plt.show()

Kp_slider = widgets.FloatSlider(value=10000, min=0, max=40000, step=100, description='Kp:')
Ki_slider = widgets.FloatSlider(value=10, min=0, max=100, step=1, description='Ki:')
ui = widgets.interactive(update_plot, Kp=Kp_slider, Ki=Ki_slider)
out = widgets.VBox([ui])
plt.close()  # Evitar mostrar el plot vacío inicial
display(out)
